In [1]:
# Group project for Credit Card Fraud Detection with TensorFlow
# Carolina Carvalho Manhaes Leite (leite2)
# Khavya Chandrasekaran (khavyac2)
# Xuehui Chao (xuehuic2)

# IMPORTING LIBRARIES
from tensorflow import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from keras.models import Sequential
from keras.layers import Activation, Dense
from sklearn.metrics import confusion_matrix, classification_report

# IMPORTING DATA FROM GOOGLE DRIVE
from google.colab import drive 
drive.mount('/content/gdrive')

# READING AND MANIPULATING THE CSV FILE
df=pd.read_csv('gdrive/My Drive/creditcard.csv')
print('The complete dataset has ' + str(df.shape[0]) + ' rows and ' + str(df.shape[1]) + ' columns.\n')

# Drop the target (Class) from the features array
X = df.drop('Class', axis=1).values
print('The features array has ' + str(X.shape[0]) + ' rows and ' + str(X.shape[1]) + ' columns.')
# Extract the target (Class) and move it to its own array
y = df['Class'].values
y = y.reshape(-1, 1)
print('The target array has ' + str(y.shape[0]) + ' rows.')

# DESCRIPTIVE STATISTICS #

# DS of Target Variable
print ("Descritive Statistics of Target Variable (Class)\n")
for x in range(-1,0):
    lastvar = df[df.columns[x]]
    print (lastvar.describe())
    print(" ")
print("------------\n")

# DS of the Features
print ("Descriptive Statistics of the Features\n")
for x in range(0, 30):
    lastvar = df[df.columns[x]]
    print (lastvar.describe())
    print(" ")
print("------------\n")

print(df.tail())
print(df['Class'].value_counts()/len(df) * 100,2)
print(df['Class'].value_counts())


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
The complete dataset has 284807 rows and 31 columns.

The features array has 284807 rows and 30 columns.
The target array has 284807 rows.
Descritive Statistics of Target Variable (Class)

count    284807.000000
mean          0.001727
std           0.041527
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Class, dtype: float64
 
------------

Descriptive Statistics of the Features

count 

In [3]:
# RESAMPLING THE DATA (WE HAVE AN UNBALANCED TARGET VARIABLE: 0.17% FOR FRAUD)

# Separating majority and minority classes
df_majority = df[df.Class==0]
df_minority = df[df.Class==1]
 
# Upsampling minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # Sample with replacement
                                 n_samples=284315, # 50%/50% resampled
                                 random_state=123) # Seed
 
# Combining majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Displaying new class counting
print(df_upsampled['Class'].value_counts())
print("")

# Dropping the target (Class) from the features array
X = df_upsampled.drop('Class', axis=1).values
y = df_upsampled['Class'].values
y = y.reshape(-1, 1)

# Splitting train and test datasets
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

1    284315
0    284315
Name: Class, dtype: int64



In [4]:
# NEURAL NETWORK

# CLEARING KERAS SESSION
keras.backend.clear_session()

# Initializing the model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim = X_train.shape[1], kernel_initializer='random_normal'))
model.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

# Specifying the training configuration (optimizer, loss, metrics)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Fitting model on the training dataset
model.fit(X_train, y_train, epochs = 10, verbose = 1)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





454904/454904 [==============================] - 22s 49us/step - loss: 0.6074 - acc: 0.7447
Epoch 2/10
454904/454904 [==============================] - 21s 46us/step - loss: 0.1956 - acc: 0.9316
Epoch 3/10
454904/454904 [==============================] - 21s 47us/step - loss: 0.1874 - acc: 0.9346
Epoch 4/10
454904/454904 [==============================] - 21s 46us/step - loss: 0.1850 - acc: 0.9373
Epoch 5/10
454904/454904 [==============================] - 21s 46us/step - loss: 0.1852 - acc: 0.9379
Epoch 6/10
454904/454904 [==============================] - 21s 46us/step - loss: 0.1894 - acc: 0.9367
Epoch 7/10
454904/454904 [==============================] - 22s 48us/step - loss: 0.1779 - acc: 0.9404
Epoch 8/10
454904/454904 [==============================] - 22s 49us/step - loss: 0.1823 - acc: 0.9412
Epoch 9/10
454904/454904 [==============================] - 22s 49us/s

In [7]:
# PERFORMANCE METRICS

# Fitting the model on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)
# Confusion matrix
print(confusion_matrix(y_test, y_pred))
# Classification report
print(classification_report(y_test, y_pred))

[[56106   640]
 [ 5961 51019]]
              precision    recall  f1-score   support

           0       0.90      0.99      0.94     56746
           1       0.99      0.90      0.94     56980

    accuracy                           0.94    113726
   macro avg       0.95      0.94      0.94    113726
weighted avg       0.95      0.94      0.94    113726

